In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import random
import re
import string
import tensorflow_text as tf_text

import tensorflow as tf
from sklearn.model_selection import train_test_split

In [5]:
#train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
extra_train = pd.read_csv("/kaggle/input/daigt-external-dataset/daigt_external_dataset.csv")
#extra_train_new = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv")
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
#train_prompts = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')

In [6]:
extra_train_1 = pd.read_csv("/kaggle/input/llm-mistral-7b-instruct-texts/Mistral7B_CME_v7.csv").drop(['prompt_id','prompt_name'], axis=1)
extra_train_2 = pd.read_csv("/kaggle/input/hello-claude-1000-essays-from-anthropic/persuade15_claude_instant1.csv").drop(['prompt_id','essay_title'],axis=1).rename(columns = {'essay_text': 'text'})
extra_train_2['generated'] = 1
extra_train_3 = pd.read_csv("/kaggle/input/daigt-data-llama-70b-and-falcon180b/llama_falcon_v3.csv").drop(['prompt_name','model'], axis=1)                           
extra_train_4 = pd.read_csv("/kaggle/input/llm-generated-essay-using-palm-from-google-gen-ai/LLM_generated_essay_PaLM.csv").drop(['id','prompt_id'], axis=1)

extra_train_5 = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv").rename(columns = {'label': 'generated'}).drop(['prompt_name','source', 'RDizzl3_seven'], axis=1)

extra_train_5 = extra_train_5[extra_train_5['generated'] == 0]
extra_train_5 = extra_train_5.sample(n=18149)

extra_train_6 = pd.read_csv("/kaggle/input/4k-mixtral87b-crafted-essays-for-detect-ai-comp/Mixtral8x7b_4k_essays_for_DetectAIGeneratedTextCompetition.csv").rename(columns = {'AI_Essay': 'text'}).drop(['model_Raw_output','student_id','prompt_id','index'], axis=1)
extra_train_6['generated'] = 1

In [7]:
extra_train_new = pd.concat([extra_train_1, extra_train_2,
                             extra_train_3, extra_train_4,
                             extra_train_5, extra_train_6], axis=0)

In [8]:
extra_train_student_generated = extra_train[['text']]
extra_train_student_generated['generated'] = 0

extra_train_ai_generated = extra_train[['source_text']].rename(columns = {'source_text':'text'})
extra_train_ai_generated['generated'] = 1

extra_train_f = pd.concat([extra_train_student_generated, extra_train_ai_generated, extra_train_new], axis=0)
#extra_train_f = pd.concat([extra_train_student_generated, extra_train_ai_generated], axis=0)
extra_train_f

/tmp/ipykernel_42/4246250905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extra_train_student_generated['generated'] = 0


,text,generated
0,Some schools in United States ofter classes fr...,0.0
1,"Four-day work week, a remarkable idea to conse...",0.0
2,Students and their families should consider an...,0.0
3,Agree you will never grow if something beyond ...,0.0
4,I think our character traits are formed by inf...,0.0
...,...,...
3860,"Dear Senator,\n\nHave you ever wondered why th...",1.0
3861,There are many reasons to consider limiting ca...,1.0
3862,"Dear State Senator,\n\nI'm writing to argue in...",1.0
3863,As commuters navigate congested roads and inha...,1.0


In [10]:
external_data=extra_train_f
external_data

,text,generated
0,Some schools in United States ofter classes fr...,0.0
1,"Four-day work week, a remarkable idea to conse...",0.0
2,Students and their families should consider an...,0.0
3,Agree you will never grow if something beyond ...,0.0
4,I think our character traits are formed by inf...,0.0
...,...,...
3860,"Dear Senator,\n\nHave you ever wondered why th...",1.0
3861,There are many reasons to consider limiting ca...,1.0
3862,"Dear State Senator,\n\nI'm writing to argue in...",1.0
3863,As commuters navigate congested roads and inha...,1.0


In [11]:
external_data.to_csv('external_data.csv',index=False)

generated
0.0    20570
1.0    20570
Name: count, dtype: int64